In [28]:
import numpy as np

import scipy.linalg as la
import scipy.sparse as sparse

%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.cm as cm

# Problem 8
$$ - \left( \frac{\partial^2 u}{\partial x^2} + \frac{\partial^2 u}{\partial y^2} \right) - e^u = 0$$
Leads to the FD equation
$$4u_{i,j} - u_{i-1,j} - u_{i+1,j} - u_{i,j-1} - u_{i,j+1} - h^2e^{u_{i,j}} = 0$$
for $ 0< i,j < n$
$$f(\vec{u}) = A\vec{u} - e^\vec{u} = 0$$
So the Jacobian is given by the following formulae
$$
\frac{\partial f}{\partial u_{i,j}} = 4 - h^2e^{u_{i,j}}
$$

In [81]:
N = 2**7-1
n = N**2
h = 1/(N+1)

a = .1
iter_max = 10**5
tol = 10**-6


J = np.diag([4]*N) + np.diag([-1]*(N-1),k=-1) + np.diag([-1]*(N-1),k=1)
K = np.diag([-1]*N)
shape_matrix = np.diag([1]*(N-1),k=-1) + np.diag([1]*(N-1),k=1)
A = np.kron(shape_matrix,K) + np.kron(np.eye(N),J)

# generate initial u
X = np.linspace(h,1-h,N)
u = np.array([ a*x*(1-x)*(1-y) for x in X for y in X]).reshape((n,1))

#u = np.ones((n,1))
Jac = A - np.diag(h**2*np.exp(u).ravel())
u_new = u + la.solve(Jac, h**2*np.exp(u) - np.dot(A,u))
for iteration in range(1,iter_max+1):
    if la.norm(u_new-u) < tol*n:
        break
    u = u_new
    Jac = A - np.diag(h**2*np.exp(u).ravel())
    u_new = u + la.solve(Jac, h**2*np.exp(u) - np.dot(A,u))
    
print(iteration)
X, Y = np.meshgrid(np.linspace(h,1-h,N),np.linspace(h,1-h,N))
U = u.reshape((N,N))
plt.pcolormesh(X,Y,U, cmap=cm.coolwarm)

KeyboardInterrupt: 

In [3]:
def conjugate_gradient(A, b, x_0, tol=10**-2, max_iter=10**3 ):
    x = x_0
    r = b - A@x
    delta = np.dot(r.T,r)
    b_delta = np.dot(b.T,b)
    p = r
    for k in range(max_iter):
        if delta < b_delta * tol**2:
            break
        s = A@p
        alpha = delta/(np.dot(p.T,s))
        x_new = x + alpha*p
        r -= alpha*s
        delta_new = np.dot(r.T,r)
        p = r + delta_new/delta * p
        x, delta = x_new, delta_new
    return x, k

In [65]:
def sp_conjugate_gradient(A, b, x_0, tol=10**-2, max_iter=10**3 ):
    x = x_0
    r = b - A.dot(x)
    delta = np.dot(r.T,r)
    cutoff = np.dot(b.T,b)
    b_delta = np.dot(b.T,b) * tol**2
    p = r
    for k in range(max_iter):
        if delta < cutoff: #b_delta * tol**2:
            break
        s = A.dot(p)
        alpha = delta/(np.dot(p.T,s))
        #print(type(alpha))
        #print(type(p))
        #print((p*alpha).shape)
        x_new = x + p*alpha
        r -= s*alpha
        delta_new = np.dot(r.T,r)
        p = r + p*delta_new/delta
        x, delta = x_new, delta_new
    return x, k

In [75]:
N = 2**4-1
n = N**2
h = 1/(N+1)

a = .1
iter_max = 10**0
tol = 10**-6


J = np.diag([4]*N) + np.diag([-1]*(N-1),k=-1) + np.diag([-1]*(N-1),k=1)
K = np.diag([-1]*N)
shape_matrix = np.diag([1]*(N-1),k=-1) + np.diag([1]*(N-1),k=1)
A = np.kron(shape_matrix,K) + np.kron(np.eye(N),J)
A = sparse.lil_matrix(A)

# generate initial u
X = np.linspace(h,1-h,N)
u = np.array([ a*x*(1-x)*(1-y) for x in X for y in X]).reshape((n,1))

#u = np.ones((n,1))
Jac = A - np.diag(h**2*np.exp(u).ravel())
u_new, k = sp_conjugate_gradient(Jac, h**2*np.exp(u) - A.dot(u), u, tol=10**-2, max_iter=1)
for iteration in range(1,iter_max+1):
    if la.norm(u_new-u) < tol*n:
        break
    u = u_new
    Jac = A - np.diag(h**2*np.exp(u).ravel())
    u_new, k = sp_conjugate_gradient(Jac, h**2*np.exp(u) - A.dot(u), u, tol=10**-2, max_iter=1)
    
print(iteration)
X, Y = np.meshgrid(np.linspace(h,1-h,N),np.linspace(h,1-h,N))
U = u.reshape((N,N))
plt.pcolormesh(X,Y,U, cmap=cm.coolwarm)

1


Error in callback <function install_repl_displayhook.<locals>.post_execute at 0x7f3967ea22f0> (for post_execute):


ValueError: Collections can only map rank 1 arrays

ValueError: Collections can only map rank 1 arrays

In [78]:
U

matrix([[ 0.00081268,  0.00643429,  0.00611611,  0.00579794,  0.00547977,
          0.0051616 ,  0.00484342,  0.00452526,  0.00420709,  0.00388892,
          0.00357075,  0.00325258,  0.00293442,  0.00261625,  0.00229809],
        [ 0.00044469,  0.01089543,  0.01025859,  0.00962176,  0.00898492,
          0.00834809,  0.00771126,  0.00707444,  0.00643761,  0.00580079,
          0.00516397,  0.00452716,  0.00389035,  0.00325354,  0.00261673],
        [ 0.00013342,  0.01467035,  0.01376385,  0.01285736,  0.01195088,
          0.0110444 ,  0.01013793,  0.00923146,  0.008325  ,  0.00741855,
          0.0065121 ,  0.00560565,  0.00469921,  0.00379278,  0.00288635],
        [-0.00012118,  0.01775897,  0.01663185,  0.01550472,  0.01437761,
          0.01325051,  0.01212341,  0.01099633,  0.00986925,  0.00874218,
          0.00761511,  0.00648806,  0.00536101,  0.00423398,  0.00310695],
        [-0.00031916,  0.02016128,  0.01886254,  0.01756381,  0.0162651 ,
          0.01496639,  0.01366769,

In [40]:
type(A.dot(u))

numpy.ndarray

In [60]:
u* 1/u.T.dot(u)

array([[0.21264185],
       [0.19846573],
       [0.18428961],
       [0.17011348],
       [0.15593736],
       [0.14176124],
       [0.12758511],
       [0.11340899],
       [0.09923286],
       [0.08505674],
       [0.07088062],
       [0.05670449],
       [0.04252837],
       [0.02835225],
       [0.01417612],
       [0.39693146],
       [0.37046936],
       [0.34400727],
       [0.31754517],
       [0.29108307],
       [0.26462097],
       [0.23815888],
       [0.21169678],
       [0.18523468],
       [0.15877258],
       [0.13231049],
       [0.10584839],
       [0.07938629],
       [0.05292419],
       [0.0264621 ],
       [0.55286882],
       [0.5160109 ],
       [0.47915298],
       [0.44229506],
       [0.40543713],
       [0.36857921],
       [0.33172129],
       [0.29486337],
       [0.25800545],
       [0.22114753],
       [0.18428961],
       [0.14743169],
       [0.11057376],
       [0.07371584],
       [0.03685792],
       [0.68045393],
       [0.63509034],
       [0.589